### Estruturando Faker

In [1]:
from faker import Faker
import random

fake = Faker('pt_BR')
palavras = []

with open("palavras.txt", "r") as palavra:
    dados = palavra.read()
    palavras = dados.split(",")
    

print(palavras)
    


def criar_campo_falso():
    campo = {
        "nome": fake.word(ext_word_list=palavras),
        "tipo": random.choice(["text", "int", "float", "datetime", "bool"]),
        "permite_nulo": fake.boolean()
    }
    return campo

def criar_tabela_falsa():
    tabela = {
        "nome": fake.word(ext_word_list=palavras),
        "campos": [criar_campo_falso() for _ in range(random.randint(1, 10))]  
    }
    return tabela

def criar_dados_falsos(nTabelas=1, tipo="CSV"):
    dados_falsos = {
        "nome": fake.word(ext_word_list=palavras),
        "extencao_do_arquivo": tipo,
        "tabelas": [criar_tabela_falsa() for _ in range(nTabelas)]  # Gere 2 tabelas falsas
    }
    return dados_falsos



['aurélio', ' amor', ' anuência', ' análogo', ' alienado', ' abstrato', ' apogeu', ' ardiloso', ' altruísmo', ' apático', ' ascensão', ' altruísta', ' acepção', ' arbitrário', ' aleatório', ' astúcia', ' auspicioso', ' audácia', ' afeto', ' algoz', ' analogia', ' apologia', ' ativista', ' alusão', ' anseio', ' alicerce', ' aspecto', ' alheio', ' alegria', ' assim', ' anátema', ' analítico', ' abnegado', ' amálgama', ' afável', ' anacrônico', ' aquém', ' ademais', ' autonomia', ' autóctone', ' adorno', ' animosidade', ' audaz', ' almejar', ' adesão', ' através', ' aferir', ' astuto', ' autêntico', ' antagonismo', ' arrogante', ' ação', ' alcunha', ' ambíguo', ' austero', ' arcabouço', ' amizade', ' adjacente', ' ambição', ' apreço', ' alteridade', ' abstrair', ' alento', ' após', ' anexo', ' ambiguidade', ' aquisição', ' ambicioso', ' amigo', ' auferir', ' ansioso', ' abster', ' alienação', ' analisar', ' arquétipo', ' alçada', ' altivez', ' austeridade', ' aprovisionado', ' assento', '

### Configurando o Selenium

In [2]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

path_chromedriver = "/home/thiago514/Documentos/selenium-chrome/chromedriver-linux64/chromedriver-linux64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = "/home/thiago514/Documentos/selenium-chrome/chrome-linux64/chrome"


driver = webdriver.Chrome(service=Service(path_chromedriver), options=chrome_options)

### Gerando SQL de update

In [11]:
import pandas as pd

dados = pd.read_csv("dados_usuarios_com_permissao.csv")

lista_matricula = dados.matricula.to_list()

for m in lista_matricula[7:]:
    print("matricula like '" + str(m) + "' or")


matricula like '53053' or
matricula like '54054' or
matricula like '55055' or
matricula like '56056' or
matricula like '57057' or
matricula like '58058' or
matricula like '59059' or


### Fazendo Login e encaminhado para o pagina de templates

In [17]:
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep

def logar_usuario():
    list_users = pd.read_csv("dados_usuarios_com_permissao.csv")

    driver.get("http://localhost:5173/")
    sleep(0.5)
    campo_matricula = driver.find_element(By.XPATH, """//*[@id="root"]/div/div/div[1]/input""")
    campo_senha = driver.find_element(By.XPATH, """//*[@id="root"]/div/div/div[2]/input""")
    botao_entrar = driver.find_element(By.XPATH, """//*[@id="root"]/div/div/div[3]/button[1]""")

    user = list_users.iloc[random.randint(0, len(list_users) - 1)]

    campo_matricula.send_keys(str(user.matricula))
    campo_senha.send_keys(str(user.senha))
    sleep(0.5)
    botao_entrar.click()
    sleep(0.5)

    driver.get("http://localhost:5173/cadastro-template")
    sleep(0.5)



In [14]:
from selenium.webdriver.support.ui import Select

def preencheCampos(tabela):
    adicionar_campo = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[2]/button""")
    for campo in range(len(tabela["campos"])-1):
        adicionar_campo.click()
        sleep(0.5)

    for campo in range(len(tabela["campos"])):
        campo_nome = driver.find_element(By.XPATH, str.format("""//*[@id="root"]/main/form/div[2]/div/div[{n}]/input""", n=campo+1))
        campo_nulo = driver.find_element(By.XPATH, str.format("""//*[@id="root"]/main/form/div[2]/div/div[{n}]/div/input""", n=campo+1))
        campo_tipo = Select(driver.find_element(By.XPATH, str.format("""//*[@id="root"]/main/form/div[2]/div/div[{n}]/select""", n=campo+1)))

        campo_nome.send_keys(tabela["campos"][campo]["nome"])
        if(tabela["campos"][campo]["permite_nulo"]):
            campo_nulo.click()
        
        campo_tipo.select_by_value(tabela["campos"][campo]["tipo"])
        sleep(0.5)

### Criando um template Falso

In [15]:
import json

def criando_template():
    num_tabelas = 1
    tipo = random.choice(["CSV", "XLSX", "XLS"])
    if(tipo != "CSV"):
        num_tabelas = random.randint(1, 5)

    template = criar_dados_falsos(nTabelas=num_tabelas, tipo=tipo)
    print(template)
    driver.get("http://localhost:5173/cadastro-template")
    sleep(0.5)

    butao_adicionar_tabela = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[1]/button""")

    for i in range(len(template["tabelas"])-1):
        butao_adicionar_tabela.click()
        sleep(0.5)

    campos_nome_tabelas = []

    for i, tabela in enumerate(template["tabelas"]):
        tabela_input = driver.find_element(By.XPATH, str.format("""//*[@id="root"]/main/form/div[1]/div/div[{n}]/input""", n=i+1))
        tabela_input.send_keys(tabela["nome"])
        campos_nome_tabelas.append(tabela_input)
        sleep(0.5)
        preencheCampos(tabela)

    tipo_csv = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[3]/div[1]/div[1]/input""")
    tipo_xlsx = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[3]/div[1]/div[2]/input""")
    tipo_xls = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[3]/div[1]/div[3]/input""")

    if template["extencao_do_arquivo"] == "CSV":
        tipo_csv.click()
    elif template["extencao_do_arquivo"] == "XLSX":
        tipo_xlsx.click()
    else:
        tipo_xls.click()

    nome_template = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[3]/div[2]/input""")

    nome_template.send_keys(template["nome"])

    salvar_botao = driver.find_element(By.XPATH, """//*[@id="root"]/main/form/div[4]/button""")
    salvar_botao.click()
    with open("templates/" + template["nome"] + ".json", "w") as f:
        json.dump(template, f)
    sleep(0.5)


### For para gerar templates

In [20]:

for i in range(100):
    logar_usuario()
    criando_template()
    sair_button = driver.find_element(By.XPATH, """//*[@id="root"]/header/a""")
    sair_button.click()
    sleep(0.5)

{'nome': ' eminência', 'extencao_do_arquivo': 'CSV', 'tabelas': [{'nome': ' transformação', 'campos': [{'nome': ' ser', 'tipo': 'text', 'permite_nulo': True}, {'nome': ' ouvir', 'tipo': 'bool', 'permite_nulo': False}]}]}
{'nome': ' monotonia', 'extencao_do_arquivo': 'XLSX', 'tabelas': [{'nome': ' subjetivo', 'campos': [{'nome': ' segmento', 'tipo': 'datetime', 'permite_nulo': False}, {'nome': ' mudança', 'tipo': 'text', 'permite_nulo': False}, {'nome': ' coercitivo', 'tipo': 'int', 'permite_nulo': False}, {'nome': ' alcunha', 'tipo': 'int', 'permite_nulo': False}, {'nome': ' krill', 'tipo': 'text', 'permite_nulo': False}]}, {'nome': ' habitar', 'campos': [{'nome': ' humilde', 'tipo': 'datetime', 'permite_nulo': False}, {'nome': ' kelvin', 'tipo': 'text', 'permite_nulo': False}, {'nome': ' xofrango', 'tipo': 'float', 'permite_nulo': False}, {'nome': ' dinamismo', 'tipo': 'text', 'permite_nulo': False}, {'nome': ' abominável', 'tipo': 'bool', 'permite_nulo': True}, {'nome': ' comorbidade

WebDriverException: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"Not attached to an active page"}
  (Session info: chrome=118.0.5993.70)
Stacktrace:
#0 0x55f2a3b27fb3 <unknown>
#1 0x55f2a37fb4a7 <unknown>
#2 0x55f2a37e3aed <unknown>
#3 0x55f2a37e1f13 <unknown>
#4 0x55f2a37e235a <unknown>
#5 0x55f2a37e0dc1 <unknown>
#6 0x55f2a37ecd24 <unknown>
#7 0x55f2a387e0d5 <unknown>
#8 0x55f2a3864713 <unknown>
#9 0x55f2a383718b <unknown>
#10 0x55f2a3837f7e <unknown>
#11 0x55f2a3aed8d8 <unknown>
#12 0x55f2a3af1800 <unknown>
#13 0x55f2a3afbcfc <unknown>
#14 0x55f2a3af2418 <unknown>
#15 0x55f2a3abf42f <unknown>
#16 0x55f2a3b164e8 <unknown>
#17 0x55f2a3b166b4 <unknown>
#18 0x55f2a3b27143 <unknown>
#19 0x7f6e0e494ac3 <unknown>


In [ ]:
logar_usuario()